In [8]:
import pandas as pd
import numpy as np
from functools import reduce
import os

### Only Once

In [ ]:
from dotenv import load_dotenv, find_dotenv

# Load secrets into os.environ
load_dotenv(find_dotenv())

### Run All Together:

In [ ]:
import requests
from simple_salesforce import Salesforce
import os
import pandas as pd  # don’t forget this

def load_salesforce_data():
    """
    Connects to Salesforce using environment variables for credentials,
    fetches Contact records with fields 'hjbc_id__c' and 'marketing_allowed__c',
    handles pagination, and returns a cleaned pandas DataFrame.
    """

    # Read from already-loaded environment
    sf_username = os.environ["SF_USERNAME"]
    sf_password = os.environ["SF_PASSWORD"]
    sf_security_token = os.environ["SF_SECURITY_TOKEN"]

    # Authenticate with Salesforce
    sf = Salesforce(
        username=sf_username,
        password=sf_password,
        security_token=sf_security_token,
    )

    # Set up base instance URL and headers for raw HTTP requests
    instance_url = sf.base_url.split('/services/data')[0]
    headers = {
        'Authorization': f'Bearer {sf.session_id}',
        'Content-Type': 'application/json'
    }

    # SOQL query
    contact_query = """
        SELECT hjbc_id__c, marketing_allowed__c
        FROM Contact
    """.strip()

    url = f"{instance_url}/services/data/v57.0/query?q={contact_query}"
    response = requests.get(url, headers=headers)

    contact_data = []

    while response.status_code == 200:
        data = response.json()
        records = data.get("records", [])

        for record in records:
            contact_data.append({
                "HJBC ID": record.get("hjbc_id__c"),
                "Marketing Allowed": record.get("marketing_allowed__c", "N/A")
            })

        if not data.get("done", True):
            next_url = f"{instance_url}{data['nextRecordsUrl']}"
            response = requests.get(next_url, headers=headers)
        else:
            break

    return pd.DataFrame(contact_data)

In [25]:
def process_game_attendance(game_dfs):
    """
    Processes a list of game DataFrames into one wide-format DataFrame.
    Filters to include only "members" — season ticket holders and linked single ticket holders.

    Args:
        game_dfs (list of DataFrame): Each DataFrame represents one game's attendance.

    Returns:
        DataFrame: Wide-format with one column per game and attendance values per user.
    """
    def clean_game(df):
        rtl = "\u200F"
        df = df.copy()

        # Normalize seat info
        for col in ['Area', 'Row', 'Number']:
            df[col] = df[col].astype(str).str.replace(r'[\u200E\u200F]', '', regex=True).str.strip()

        # Add derived columns
        df['Full Name'] = df['First name'].astype(str) + " " + df['Last name'].astype(str)
        df['Sit Area'] = rtl + " " + df['Area'] + " שורה " + df['Row'] + " כיסא " + df['Number']
        df['Id'] = df['Id'].astype(str)
        df['User Id'] = df['User Id'].fillna(0).astype(int)

        # Create normalized columns
        df['Source'] = df.apply(lambda row: "tickets" if row['Type'] == 'Ticket' and pd.isna(row['STRefID']) else "members", axis=1)
        df['Normalized Id'] = df.apply(lambda row: row['Id'] if pd.isna(row['STRefID']) else str(row['STRefID']), axis=1)

        # Keep only "members"
        df = df[df['Source'] == 'members']

        # Map attendance and reshape
        df['Attendance'] = df['Attendance'].map({'Yes': True, 'No': False}).where(df['Attendance'].notna(), None)
        event_name = df['Event name'].iloc[0]

        short = df[['Full Name', 'User Id', 'Normalized Id', 'Discount rule', 'Attendance']].drop_duplicates()
        short = short.rename(columns={'Attendance': event_name, 'Normalized Id': 'Id'})

        return short

    cleaned_games = [clean_game(df) for df in game_dfs]

    # Merge all event columns into wide format
    result_df = reduce(lambda left, right: pd.merge(left, right, on=['User Id', 'Id'], how='outer'), cleaned_games)

    result_df = result_df.where(pd.notnull(result_df), None)

    # Reorder with 'Full Name' first
    if 'Full Name' in result_df.columns:
        cols = ['Full Name'] + [c for c in result_df.columns if c != 'Full Name']
        result_df = result_df[cols]

    return result_df

In [26]:
def create_attendance_summary(members, game_dfs, singles):
    today = pd.Timestamp('today')
    members['Birth date'] = pd.to_datetime(members['Birth date'], errors='coerce')
    members['Age'] = members['Birth date'].apply(
        lambda x: today.year - x.year - ((today.month, today.day) < (x.month, x.day)) if pd.notnull(x) else None
    )

    members = members[[
        'Fan / Company', 'Id', 'User Id', 'Phone', 'Email', 'City', 'Gender', 'Age',
        'Seat', 'Date.1', 'Voucher number', 'Voucher value', 'Base price', 'Price', 'Status'
    ]].copy()

    # Merge Salesforce data
    sf_data = load_salesforce_data()
    members = members[members['Status'] == 'Active'].copy()
    members['User Id'] = members['User Id'].astype(str)
    sf_data['HJBC ID'] = sf_data['HJBC ID'].astype(str)
    sf_merged = pd.merge(members, sf_data, left_on='User Id', right_on='HJBC ID', how='left').drop(columns=['HJBC ID'])

    # Process wide-format attendance table
    merged_games = process_game_attendance(game_dfs)
    sf_merged['Id'] = pd.to_numeric(sf_merged['Id'], errors='coerce').astype('Int64')
    merged_games['Id'] = pd.to_numeric(merged_games['Id'], errors='coerce').astype('Int64')

    sf_merged['User Id'] = sf_merged['User Id'].astype(str)
    merged_games['User Id'] = merged_games['User Id'].astype(str)

    all_data = pd.merge(sf_merged, merged_games, on=['Id', 'User Id'], how='left')

    # Merge with singles (also wide-format)
    all_data['User Id'] = all_data['User Id'].astype(str)
    singles['User Id'] = singles['User Id'].astype(str)
    combined = pd.merge(all_data, singles, on='User Id', how='outer', suffixes=('', '_singles'))

    # For each event, fill missing season ticket attendance with singles attendance
    event_columns = [col for col in merged_games.columns if col != 'Id']
    for col in event_columns:
        col_s = f'{col}_singles'
        if col_s in combined.columns:
            combined[col] = combined[col].combine_first(combined[col_s])

    # Drop extra '_singles' columns
    combined = combined[[col for col in combined.columns if not col.endswith('_singles')]]

    # Count games attended
    combined['Games Attended'] = combined[event_columns].apply(lambda row: sum(x is True for x in row), axis=1)
    combined['Attendance Ratio'] = (combined['Games Attended'] / len(event_columns) * 100).round(1).astype(float)

    combined = combined.where(pd.notnull(combined), None)

    return combined

### Run it for all the games:

In [27]:
def add_game_to_summary(summary_df, members, new_game_df):
    event_column = [
        col for col in new_game_df.columns 
        if col not in ['Id', 'User Id', 'Full Name']
    ][0]

    # Normalize
    members = members[members['Status'] == 'Active'].copy()
    members['User Id'] = members['User Id'].astype(str)
    members['Id'] = pd.to_numeric(members['Id'], errors='coerce').astype('Int64')
    new_game_df['User Id'] = new_game_df['User Id'].astype(str)
    new_game_df['Id'] = pd.to_numeric(new_game_df['Id'], errors='coerce').astype('Int64')

    all_data = pd.merge(members, new_game_df[['Id', 'User Id', event_column]], on=['Id', 'User Id'], how='left')

    if event_column in summary_df.columns:
        print(f"⚠️ Event '{event_column}' already in summary — skipping.")
    else:
        # Drop old metrics
        summary_df = summary_df.drop(columns=['Games Attended', 'Attendance Ratio'], errors='ignore')

        # Merge in new event
        summary_df = pd.merge(summary_df, all_data[['Id', event_column]], on='Id', how='left')

        # Recalculate metrics
        event_columns = [col for col in summary_df.columns if col.startswith('Round') or col.startswith('🏠') or 'מחזור' in col]
        summary_df['Games Attended'] = summary_df[event_columns].fillna(False).infer_objects(copy=False).astype(bool).sum(axis=1)
        summary_df['Attendance Ratio'] = (summary_df['Games Attended'] / len(event_columns) * 100).round(1)

        # Reorder columns
        first_cols = [col for col in summary_df.columns if col not in ['Games Attended', 'Attendance Ratio']]
        summary_df = summary_df[first_cols + ['Games Attended', 'Attendance Ratio']]
        summary_df = summary_df.drop(columns='User Id (from wide)', errors='ignore')

    return summary_df

In [28]:
# Load base data
members = pd.read_csv('List_Report_20250609080140.csv')

# Try to load previously saved data
if os.path.exists('processed_games.pkl') and os.path.exists('summary_df.pkl'):
    print("Loading previously saved data...")
    processed_games = pd.read_pickle('processed_games.pkl')
    summary_df = pd.read_pickle('summary_df.pkl')
else:
    print("No previous data found, starting fresh.")
    processed_games = None
    summary_df = None

game_files = [
    'AuthorizedAttendanceReportCsv_Round 2_ Hapoel Tel Aviv 🚗.csv',
    'AuthorizedAttendanceReportCsv_Round 4_ Hapoel Holon 🏠.csv',
    'AuthorizedAttendanceReportCsv_Round 8_ Hapoel Afula 🏠.csv',
    'AuthorizedAttendanceReportCsv_🏠 מחזור 9_ הפועל ״בנק יהב״ ירושלים -הפועל גליל עליון.csv',
    'AuthorizedAttendanceReportCsv_🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת"א.csv',
    # 'AuthorizedAttendanceReportCsv_🏠 מחזור 13 (רדיוס בחולון)_  מכבי עירוני רמת גן.csv',
    'AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 17_ גלבוע גליל 🏠.csv',
    'AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 18_ הרצליה 🏠 .csv',
    'AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 20_ הפועל חיפה 🏠.csv',
    'AuthorizedAttendanceReportCsv_מחזור 22_ אליצור עירוני נתניה 🏠.csv',
    'AuthorizedAttendanceReportCsv_מחזור 24_ הפועל באר שבע - דימונה 🏠.csv',
    'AuthorizedAttendanceReportCsv_מחזור 26_ עירוני נס ציונה 🏠.csv',
    'AuthorizedAttendanceReportCsv_רבע גמר 1_ מכבי  עירוני רמת גן 🏠.csv',
    'AuthorizedAttendanceReportCsv_רבע גמר 3_ מכבי  עירוני רמת גן 🏠.csv',
    'AuthorizedAttendanceReportCsv_חצי גמר משחק 2_ הפועל תל אביב 🏠.csv',
    'AuthorizedAttendanceReportCsv_גמר ליגת ווינר משחק 2_ מכבי תל אביב 🏠.csv'
]

# Process each new game
for file in game_files:
    print(f"\nProcessing {file}...")
    game_df = pd.read_csv(file)
    wide_df = process_game_attendance([game_df])

    # Normalize
    wide_df['Id'] = pd.to_numeric(wide_df['Id'], errors='coerce').astype('Int64')
    wide_df['User Id'] = wide_df['User Id'].astype(str)

    if processed_games is None:
        # First game being processed
        processed_games = wide_df.copy()
        summary_df = create_attendance_summary(members, [game_df], singles=processed_games)
        print("Initialized processed_games and summary_df.")
    else:
        processed_games['Id'] = pd.to_numeric(processed_games['Id'], errors='coerce').astype('Int64')
        processed_games['User Id'] = processed_games['User Id'].astype(str)

        # Detect new columns
        new_cols = [col for col in wide_df.columns if col not in ['Id', 'User Id', 'Full Name'] and col not in processed_games.columns]
        
        if new_cols:
            trimmed = wide_df[['Id', 'User Id'] + new_cols]
            processed_games = pd.merge(processed_games, trimmed, on=['Id', 'User Id'], how='outer')
            summary_df = add_game_to_summary(summary_df, members, wide_df)
            print(f"Appended new event(s): {new_cols}")
        else:
            print("Game already processed — skipping.")

# Identify event columns (exclude metadata)
event_columns_pg = [col for col in processed_games.columns if col not in ['Id', 'User Id', 'Full Name']]
event_columns_summary = [col for col in summary_df.columns if col not in ['Id', 'User Id', 'Games Attended', 'Attendance Ratio']]

# Replace NaN with None only in event columns
processed_games[event_columns_pg] = processed_games[event_columns_pg].where(pd.notnull(processed_games[event_columns_pg]), None)
summary_df[event_columns_summary] = summary_df[event_columns_summary].where(pd.notnull(summary_df[event_columns_summary]), None)


# Save final state
processed_games.to_pickle('processed_games.pkl')
summary_df.to_pickle('summary_df.pkl')
print("\nAll games processed and saved.")

Loading previously saved data...

Processing AuthorizedAttendanceReportCsv_Round 2_ Hapoel Tel Aviv 🚗.csv...
Game already processed — skipping.

Processing AuthorizedAttendanceReportCsv_Round 4_ Hapoel Holon 🏠.csv...
Game already processed — skipping.

Processing AuthorizedAttendanceReportCsv_Round 8_ Hapoel Afula 🏠.csv...
Game already processed — skipping.

Processing AuthorizedAttendanceReportCsv_🏠 מחזור 9_ הפועל ״בנק יהב״ ירושלים -הפועל גליל עליון.csv...
Game already processed — skipping.

Processing AuthorizedAttendanceReportCsv_🏠 מחזור11_ הפועל ״בנק יהב״ ירושלים -מכבי ת"א.csv...
Game already processed — skipping.

Processing AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 17_ גלבוע גליל 🏠.csv...
Game already processed — skipping.

Processing AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 18_ הרצליה 🏠 .csv...
Game already processed — skipping.

Processing AuthorizedAttendanceReportCsv_ליגת ווינר סל מחזור 20_ הפועל חיפה 🏠.csv...
Game already processed — skipping.

Processing Au

/var/folders/nt/lqq9984n7p3fpfvd92swmw280000gn/T/ipykernel_6367/1235776227.py:36: DtypeWarning: Columns (16,18,21,27,35,45,65) have mixed types. Specify dtype option on import or set low_memory=False.
  game_df = pd.read_csv(file)


In [29]:
summary_df = summary_df.drop_duplicates(subset='Id')
print('Shape:', summary_df.shape)
summary_df

Shape: (4341, 34)


,Fan / Company,Id,User Id,Phone,Email,City,Gender,Age,Seat,Date.1,...,ליגת ווינר סל מחזור 20: הפועל חיפה 🏠,מחזור 22: אליצור עירוני נתניה 🏠,מחזור 24: הפועל באר שבע - דימונה 🏠,מחזור 26: עירוני נס ציונה 🏠,רבע גמר 1: מכבי עירוני רמת גן 🏠,רבע גמר 3: מכבי עירוני רמת גן 🏠,חצי גמר משחק 2: הפועל תל אביב 🏠,Discount rule,Games Attended,Attendance Ratio
0,אבי שומר,855420,10001,0524445599,avi@t-s.co.il,ירושליים,Male,69.0,"Courtside / פרקט,פרקט דרומי,A,17",2024-11-05,...,True,False,False,False,False,True,None,None,5,45.5
1,אבי שומר,855419,10001,0524445599,avi@t-s.co.il,ירושליים,Male,69.0,"Courtside / פרקט,פרקט דרומי,A,16",2024-11-05,...,True,False,False,False,False,True,None,None,5,45.5
2,אבי שומר,855417,10001,0524445599,avi@t-s.co.il,ירושליים,Male,69.0,"Courtside / פרקט,פרקט דרומי,A,14",2024-11-05,...,True,False,False,False,False,True,None,None,5,45.5
3,אבי שומר,855418,10001,0524445599,avi@t-s.co.il,ירושליים,Male,69.0,"Courtside / פרקט,פרקט דרומי,A,15",2024-11-05,...,True,False,False,False,False,True,None,None,5,45.5
4,דובי גולדברג,854432,10003,0522444004,dubi.goldberg@gmail.com,ירושלים,Male,74.0,"פרקט,פרקט מרכז,1,35",2024-10-30,...,False,False,True,False,True,False,None,None,4,36.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4341,דניאל גפני,833645,49417,None,gaffnidaniel@gmail.com,ירושלים,Male,19.0,"אולם,12 אולם,12,30",2024-08-24,...,True,False,True,False,False,True,None,None,9,81.8
4342,יהונתן רסיס טל,838497,49427,None,naomirtal@gmail.com,ירושלים,Male,7.0,"גלריה,גלריה 1,3,40",2024-09-03,...,False,False,False,False,True,False,None,None,2,18.2
4343,נעם ארזי כהן,829400,49462,None,noamarazi2016@gmail.com,ירושלים,Male,17.0,"אולם,אולם 1 עמידה ,C,17",2024-08-14,...,False,True,False,False,True,True,None,None,7,63.6
4344,בעז סוננבליק,873077,49504,None,boazsonnen@gmail.com,אלעזר,Male,15.0,"אולם,אולם 6,11,32",2025-02-11,...,False,True,False,False,True,True,None,None,2,18.2


In [32]:
summary_df.columns

Index(['Fan / Company', 'Id', 'User Id', 'Phone', 'Email', 'City', 'Gender',
       'Age', 'Seat', 'Date.1', 'Voucher number', 'Voucher value',
       'Base price', 'Price', 'Status', 'Marketing Allowed', 'Full Name',
       'Round 2: Hapoel Tel Aviv 🚗', 'Round 4: Hapoel Holon 🏠',
       'Round 8: Hapoel Afula 🏠',
       '🏠 מחזור 9: הפועל ״בנק יהב״ ירושלים -הפועל גליל עליון',
       '🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא"',
       'ליגת ווינר סל מחזור 17: גלבוע גליל 🏠',
       'ליגת ווינר סל מחזור 18: הרצליה 🏠 ',
       'ליגת ווינר סל מחזור 20: הפועל חיפה 🏠',
       'מחזור 22: אליצור עירוני נתניה 🏠', 'מחזור 24: הפועל באר שבע - דימונה 🏠',
       'מחזור 26: עירוני נס ציונה 🏠', 'רבע גמר 1: מכבי  עירוני רמת גן 🏠',
       'רבע גמר 3: מכבי  עירוני רמת גן 🏠', 'חצי גמר משחק 2: הפועל תל אביב 🏠',
       'Discount rule', 'Games Attended', 'Attendance Ratio'],
      dtype='object')

In [30]:
summary_df.iloc[0]

Fan / Company                                                                   אבי שומר
Id                                                                                855420
User Id                                                                            10001
Phone                                                                         0524445599
Email                                                                      avi@t-s.co.il
City                                                                            ירושליים
Gender                                                                              Male
Age                                                                                 69.0
Seat                                                    Courtside / פרקט,פרקט דרומי,A,17
Date.1                                                                        2024-11-05
Voucher number                                                                      None
Voucher value        

In [10]:
summary_df.to_excel('attendance_summary.xlsx', index=False)

In [33]:
count_8 = (processed_games['Id'].astype(str).str.startswith('8')).sum()
count_3 = (processed_games['Id'].astype(str).str.startswith('3')).sum()

print(f"✅ IDs starting with 8: {count_8}")
print(f"✅ IDs starting with 3: {count_3}")

✅ IDs starting with 8: 6909
✅ IDs starting with 3: 0


In [34]:
print('Shape:', processed_games.shape)
processed_games

Shape: (6909, 19)


,Full Name,User Id,Id,Round 2: Hapoel Tel Aviv 🚗,Round 4: Hapoel Holon 🏠,Round 8: Hapoel Afula 🏠,🏠 מחזור 9: הפועל ״בנק יהב״ ירושלים -הפועל גליל עליון,"🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא""",ליגת ווינר סל מחזור 17: גלבוע גליל 🏠,ליגת ווינר סל מחזור 18: הרצליה 🏠,ליגת ווינר סל מחזור 20: הפועל חיפה 🏠,מחזור 22: אליצור עירוני נתניה 🏠,מחזור 24: הפועל באר שבע - דימונה 🏠,מחזור 26: עירוני נס ציונה 🏠,רבע גמר 1: מכבי עירוני רמת גן 🏠,רבע גמר 3: מכבי עירוני רמת גן 🏠,חצי גמר משחק 2: הפועל תל אביב 🏠,Discount rule,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠
0,five test,1330587,801629,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
1,Test twelve,1330653,801726,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
2,Test ROBO,1330686,801760,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
3,Testing ROBO,1330719,801793,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
4,עירית עינב,26178,801992,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6904,NaN,11567,899888,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6905,NaN,12825,899934,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6906,NaN,14075,899935,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6907,NaN,1420313,899936,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,False


### Remove users that are not in members but are in processed_games

In [35]:
processed_games['User Id'] = pd.to_numeric(processed_games['User Id'], errors='coerce').astype('Int64')
members['User Id'] = pd.to_numeric(members['User Id'], errors='coerce').astype('Int64')

processed_games_cleaned = processed_games[
    processed_games['User Id'].isin(members['User Id'])
]

processed_games_cleaned

,Full Name,User Id,Id,Round 2: Hapoel Tel Aviv 🚗,Round 4: Hapoel Holon 🏠,Round 8: Hapoel Afula 🏠,🏠 מחזור 9: הפועל ״בנק יהב״ ירושלים -הפועל גליל עליון,"🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא""",ליגת ווינר סל מחזור 17: גלבוע גליל 🏠,ליגת ווינר סל מחזור 18: הרצליה 🏠,ליגת ווינר סל מחזור 20: הפועל חיפה 🏠,מחזור 22: אליצור עירוני נתניה 🏠,מחזור 24: הפועל באר שבע - דימונה 🏠,מחזור 26: עירוני נס ציונה 🏠,רבע גמר 1: מכבי עירוני רמת גן 🏠,רבע גמר 3: מכבי עירוני רמת גן 🏠,חצי גמר משחק 2: הפועל תל אביב 🏠,Discount rule,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠
0,five test,1330587,801629,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
1,Test twelve,1330653,801726,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
2,Test ROBO,1330686,801760,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
3,Testing ROBO,1330719,801793,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
4,עירית עינב,26178,801992,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6904,NaN,11567,899888,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6905,NaN,12825,899934,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6906,NaN,14075,899935,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6907,NaN,1420313,899936,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,False


In [14]:
4483-4341

142

### Remove users that are not in summary_df but are in processed_games

In [36]:
summary_df['User Id'] = pd.to_numeric(summary_df['User Id'], errors='coerce').astype('Int64')

final_processed_game = processed_games_cleaned[
    processed_games_cleaned['User Id'].isin(summary_df['User Id'])
]
final_processed_game

,Full Name,User Id,Id,Round 2: Hapoel Tel Aviv 🚗,Round 4: Hapoel Holon 🏠,Round 8: Hapoel Afula 🏠,🏠 מחזור 9: הפועל ״בנק יהב״ ירושלים -הפועל גליל עליון,"🏠 מחזור11: הפועל ״בנק יהב״ ירושלים -מכבי תא""",ליגת ווינר סל מחזור 17: גלבוע גליל 🏠,ליגת ווינר סל מחזור 18: הרצליה 🏠,ליגת ווינר סל מחזור 20: הפועל חיפה 🏠,מחזור 22: אליצור עירוני נתניה 🏠,מחזור 24: הפועל באר שבע - דימונה 🏠,מחזור 26: עירוני נס ציונה 🏠,רבע גמר 1: מכבי עירוני רמת גן 🏠,רבע גמר 3: מכבי עירוני רמת גן 🏠,חצי גמר משחק 2: הפועל תל אביב 🏠,Discount rule,גמר ליגת ווינר משחק 2: מכבי תל אביב 🏠
0,five test,1330587,801629,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
1,Test twelve,1330653,801726,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
2,Test ROBO,1330686,801760,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
3,Testing ROBO,1330719,801793,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
4,עירית עינב,26178,801992,False,False,False,False,False,False,False,False,False,False,False,False,False,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6904,NaN,11567,899888,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6905,NaN,12825,899934,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6906,NaN,14075,899935,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True
6907,NaN,1420313,899936,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,False


In [37]:
final_processed_game.to_pickle('processed_games.pkl')
summary_df.to_pickle('summary_df.pkl')

In [38]:
processed_games.to_excel('processed_games.xlsx', index=False)
summary_df.to_excel('summary_df.xlsx', index=False)